In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
from pinecone import Pinecone,ServerlessSpec
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings



c:\Users\ACER\miniconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",dimension=768,metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1")
)


In [5]:
import json
data=json.load(open("review.json"))
data['reviews']

[{'professor': 'John Doe',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Amazing instructor! Explains complex concepts clearly.'},
 {'professor': 'Jane Smith',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Great teacher, but assignments can be tough.'},
 {'professor': 'Bob Johnson',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Very knowledgeable and helpful.'},
 {'professor': 'Alice Brown',
  'subject': 'English',
  'stars': 4,
  'review': 'Good teacher, but lectures can be dry.'},
 {'professor': 'Mike Davis',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Excellent instructor! Makes complex concepts easy to understand.'},
 {'professor': 'Emma Taylor',
  'subject': 'Chemistry',
  'stars': 4,
  'review': 'Good teacher, but lab sessions can be disorganized.'},
 {'professor': 'David Lee',
  'subject': 'History',
  'stars': 5,
  'review': 'Very engaging and knowledgeable instructor.'},
 {'professor': 'Sophia Patel',
  'subject': 'Psychology',
  'stars': 4,


In [8]:
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Loop through reviews and create embeddings
processdata = []
for review in data['reviews']:
    # Get the embedding
    embedding = embeddings_model.embed_query(review['review'])    
    # Append to process data
    processdata.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

ValidationError: 1 validation error for GoogleGenerativeAIEmbeddings
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. (type=value_error)

In [9]:
processdata[0]

{'values': [0.03798361122608185,
  -0.01409335806965828,
  0.05811094865202904,
  -0.018174905329942703,
  0.025762107223272324,
  0.02780117094516754,
  0.04583007097244263,
  0.01847224310040474,
  0.02479519322514534,
  -0.013287781737744808,
  -0.04536144807934761,
  0.024494552984833717,
  0.003721448127180338,
  -0.03769362345337868,
  0.006626592017710209,
  0.04313945770263672,
  -0.010265280492603779,
  -0.0271149892359972,
  0.01377419475466013,
  -0.019764164462685585,
  0.0017685109050944448,
  -0.027249250560998917,
  0.09194130450487137,
  -0.05195188894867897,
  0.0008018520893529058,
  -0.010377355851233006,
  0.056314192712306976,
  -0.036173272877931595,
  0.017951929941773415,
  0.06188851594924927,
  -0.049247391521930695,
  0.007353533990681171,
  -0.040714237838983536,
  0.0244054663926363,
  0.023995593190193176,
  -0.05155467614531517,
  0.040454450994729996,
  -0.016922755166888237,
  -0.015078269876539707,
  0.021835412830114365,
  0.014115991070866585,
  -0.0

In [13]:
index=pc.Index("rag")
index.upsert(
    vectors=processdata,
    namespace="ns1"
)

{'upserted_count': 20}

In [11]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}